In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%tensorflow_version 2.x
import tensorflow
print(tensorflow.__version__)

TensorFlow 2.x selected.
2.0.0


In [0]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

import pickle
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [0]:
ROOT = '/content/drive/My Drive/19 Fall/CSE 569/Project'

In [0]:
X_train = pickle.load(open(ROOT + '/cifar/X_train', mode='rb'))
print(X_train.shape)
X_test = pickle.load(open(ROOT + '/cifar/X_test', mode='rb'))
print(X_test.shape)
X_val = pickle.load(open(ROOT + '/cifar/X_val', mode='rb'))
print(X_val.shape)
y_train = pickle.load(open(ROOT + '/cifar/y_train', mode='rb'))
print(y_train.shape)
y_test = pickle.load(open(ROOT + '/cifar/y_test', mode='rb'))
print(y_test.shape)
y_val = pickle.load(open(ROOT + '/cifar/y_val', mode='rb'))
print(y_val.shape)

(32000, 32, 32, 3)
(10000, 32, 32, 3)
(8000, 32, 32, 3)
(32000, 10)
(10000, 10)
(8000, 10)


In [0]:
from tensorflow import keras

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import regularizers, optimizers

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import time

In [0]:
import random

In [0]:
for i in range(32000):
  if random.random() > 0.7:
    y_train[i] = np.array([-1,-1,-1,-1,-1,-1,-1,-1,-1,-1])

In [0]:
def custom_loss(Y, A):
  P = tensorflow.keras.backend.abs(Y)
  P = tensorflow.keras.backend.min(P, axis=1)
  # P = tensorflow.keras.backend.print_tensor(P, message="P is: ")

  Y_l = tensorflow.boolean_mask(Y, tensorflow.equal(P, 0))
  A_l = tensorflow.boolean_mask(A, tensorflow.equal(P, 0))
  # A_l = tensorflow.keras.backend.print_tensor(A_l, message="A_l is: ")
  A_l_log = tensorflow.keras.backend.log(A_l)
  A_l = tensorflow.keras.backend.flatten(A_l)
  A_l_log = tensorflow.keras.backend.flatten(A_l_log)
  A_l_log = tensorflow.keras.backend.abs(A_l_log)
  Y_l = tensorflow.keras.backend.flatten(Y_l)

  # Y_l = tensorflow.keras.backend.print_tensor(Y_l, message="Y_l flattened is: ")
  # A_l_log = tensorflow.keras.backend.print_tensor(A_l_log, message="A_l_log flattened is: ")

  L_l = A_l_log * Y_l
  L_l = tensorflow.boolean_mask(L_l, tensorflow.greater(L_l, 0))
  L_l = tensorflow.keras.backend.mean(L_l)
  # L_l = tensorflow.keras.backend.print_tensor(L_l, message="L_l is: ")

  Y_u = tensorflow.boolean_mask(Y, tensorflow.greater(P, 0))
  A_u = tensorflow.boolean_mask(A, tensorflow.greater(P, 0))
  # Y_u = tensorflow.keras.backend.print_tensor(Y_u, message="Y_u is: ")
  # A_u = tensorflow.keras.backend.print_tensor(A_u, message="A_u is: ")
  A_u_log = tensorflow.keras.backend.log(A_u)
  # A_u_log = tensorflow.keras.backend.print_tensor(A_u_log, message="A_u_log is: ")
  A_u = tensorflow.keras.backend.flatten(A_u)
  A_u_log = tensorflow.keras.backend.flatten(A_u_log)
  A_u_log = tensorflow.keras.backend.abs(A_u_log)
  L_u = A_u * A_u_log
  L_u = tensorflow.keras.backend.mean(L_u)
  # L_u = tensorflow.keras.backend.print_tensor(L_u, message="L_u is: ")

  # L = 0 - L_l - L_u
  L = L_l + L_u
  # L = tensorflow.keras.backend.print_tensor(L, message="L is: ")
  return L/1

In [0]:
# Define the model
num_filters = 32

model = Sequential()
model.add(Conv2D(num_filters, (3,3), padding='same', input_shape=(32, 32, 3)))
model.add(Activation('relu'))
model.add(Conv2D(num_filters, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(2*num_filters, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(2*num_filters, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(10, activation='softmax'))

# Summarize the model
model.summary()

# Compile the model
model.compile(optimizer='adam', loss=custom_loss, metrics=['accuracy'])

# Train the model
start = time.time()
fit_history = model.fit(X_train, y_train,
                       batch_size=128, epochs=30,
                       validation_data = (X_test, y_test),
                       verbose=1)
end = time.time()

# Training time
print("Model took %0.2f seconds to train"%(end - start))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_4 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_5 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 16, 64)       